In [8]:
%cd /nas/home/rehanahm/notebooks/testing_NER/csci_project/erc

/nas/home/rehanahm/notebooks/testing_NER/csci_project/erc


# Splitting Datasets

In [44]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from utils import get_num_classes, ErcTextDataset
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import pickle
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer


In [10]:
df = pd.read_csv('../data/transcript_data.csv')

In [11]:
df_train = df[df['Session'] < 5]
df_valid = df[(df['Session'] == 5) & (df['Utterance'] % 2 == 0)]
df_test = df[(df['Session'] == 5) & (df['Utterance'] % 2 == 1)]

In [12]:
emo_ls = list(df_train['Emotion'].unique())

In [13]:
def fn_emo(e):
    return emo_ls.index(e)

In [14]:
df_train['label'] = list(map(fn_emo,df_train['Emotion']))
df_test['label'] = list(map(fn_emo,df_test['Emotion']))
df_valid['label'] = list(map(fn_emo,df_valid['Emotion']))

/tmp/ipykernel_38941/1716027103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['label'] = list(map(fn_emo,df_train['Emotion']))
/tmp/ipykernel_38941/1716027103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['label'] = list(map(fn_emo,df_test['Emotion']))
/tmp/ipykernel_38941/1716027103.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [15]:
df_train[['Unnamed: 0','text','label']].to_csv('../data/train.csv')
df_test[['Unnamed: 0','text','label']].to_csv('../data/test.csv')
df_valid[['Unnamed: 0','text','label']].to_csv('../data/val.csv')

In [26]:
%mkdir model_custom_checkpoint

In [27]:
%cd model_custom_checkpoint

/nas/home/rehanahm/notebooks/testing_NER/csci_project/erc/model_custom_checkpoint


In [29]:
# !wget https://huggingface.co/tae898/emoberta-base/resolve/main/pytorch_model.bin
# !wget https://huggingface.co/tae898/emoberta-base/resolve/main/tokenizer_config.json.json
# !wget https://huggingface.co/tae898/emoberta-base/resolve/main/trainer_state.json
# !wget https://huggingface.co/tae898/emoberta-base/resolve/main/tokenizer.json
# !wget https://huggingface.co/tae898/emoberta-base/resolve/main/val-results.json
# !wget https://huggingface.co/tae898/emoberta-base/resolve/main/config.json

--2022-03-25 20:14:18--  https://huggingface.co/tae898/emoberta-base/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 34.200.173.213, 34.224.55.150, 34.197.58.156, ...
Connecting to huggingface.co (huggingface.co)|34.200.173.213|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/tae898/emoberta-base/f89836cedb395f1321d50bb4358e6baefe94a4d24ae15727d00e21befb2fdd5f [following]
--2022-03-25 20:14:18--  https://cdn-lfs.huggingface.co/tae898/emoberta-base/f89836cedb395f1321d50bb4358e6baefe94a4d24ae15727d00e21befb2fdd5f
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.226.219.110, 13.226.219.109, 13.226.219.35, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.226.219.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 498686637 (476M) [application/zip]
Saving to: ‘pytorch_model.bin’

100%[======================================>] 498,686,637 21.3MB

In [30]:
%cd ..

/nas/home/rehanahm/notebooks/testing_NER/csci_project/erc


In [31]:
os.listdir('model_custom_checkpoint')

['tokenizer.json',
 'pytorch_model.bin',
 'config.json',
 'trainer_state.json',
 'val-results.json']

In [34]:
model_checkpoint = '/nas/home/rehanahm/notebooks/testing_NER/csci_project/erc/model_custom_checkpoint'



NUM_CLASSES = get_num_classes('IEMOCAP')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=7
)

# Loading Raw Datasets

In [37]:
train_dataset = load_dataset('csv', data_files='../data/train.csv',split='train')
test_dataset = load_dataset('csv', data_files='../data/test.csv',split = 'train')
val_dataset = load_dataset('csv', data_files='../data/val.csv',split = 'train')

2022-03-25 20:15:22.460 WARNING builder - _create_builder_config: Using custom data configuration default-306d838a230f35bc


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /nas/home/rehanahm/.cache/huggingface/datasets/csv/default-306d838a230f35bc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


2022-03-25 20:15:23.129 WARNING builder - _create_builder_config: Using custom data configuration default-af67eea0e557a14c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /nas/home/rehanahm/.cache/huggingface/datasets/csv/default-af67eea0e557a14c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


2022-03-25 20:15:23.743 WARNING builder - _create_builder_config: Using custom data configuration default-3e5c71c536f5a0d7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /nas/home/rehanahm/.cache/huggingface/datasets/csv/default-3e5c71c536f5a0d7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


In [38]:
def tokenize_function(example):
    return tokenizer(example["text"], padding=True,truncation=True)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)


attention_mask = tokenized_train['attention_mask']+tokenized_test['attention_mask']+tokenized_val['attention_mask']
input_ids = tokenized_train['attention_mask']+tokenized_test['attention_mask']+tokenized_val['attention_mask']
index = tokenized_train['Unnamed: 0']+tokenized_test['Unnamed: 0']+tokenized_val['Unnamed: 0']


tokenized_train = tokenized_train.remove_columns(['Unnamed: 0','text'])
tokenized_test = tokenized_test.remove_columns(['Unnamed: 0','text'])
tokenized_val = tokenized_val.remove_columns(['Unnamed: 0','text'])

  0%|          | 0/6 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
embed_dict = {'index':index,'input_ids':input_ids,'attention_mask':attention_mask}
with open('../data/dump_embeddings.pkl','wb+') as f:
    pickle.dump(embed_dict,f)

In [47]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("test-trainer2")


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [124]:
trainer.

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0.1.
***** Running training *****
  Num examples = 5758
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2160


Step,Training Loss
500,1.218300
1000,0.967100


Saving model checkpoint to test-trainer2/checkpoint-500
Configuration saved in test-trainer2/checkpoint-500/config.json
Model weights saved in test-trainer2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer2/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer2/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer2/checkpoint-1000
Configuration saved in test-trainer2/checkpoint-1000/config.json
Model weights saved in test-trainer2/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer2/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer2/checkpoint-1000/special_tokens_map.json


In [ ]:
from pushnotif import PushNotif
handler = PushNotif(key = "bTk06BrKuJF-RuhBtd_EvG", event = "Notifier")
handler.send("EXECUTED")

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     './test-trainer/checkpoint-2000'
# )
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [59]:
predictions = trainer.predict(tokenized_test)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0.1.
***** Running Prediction *****
  Num examples = 810
  Batch size = 8


In [60]:
from sklearn.metrics import accuracy_score

In [61]:
import numpy as np

In [62]:
preds = [np.argmax(i) for i in predictions[0]]

In [63]:
len(predictions[0][0])

7

In [64]:
pd.Series(preds).value_counts()

1    227
3    190
2    186
0    123
4     84
dtype: int64

In [65]:
df_test['label'].value_counts()

2    232
3    184
1    183
0    123
4     88
Name: label, dtype: int64

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
accuracy_score(preds, df_test['label'])

0.6555555555555556

In [68]:
from sklearn.metrics import classification_report


In [69]:
print(classification_report(df_test['label'], preds, target_names=emo_ls))

              precision    recall  f1-score   support

         sad       0.69      0.69      0.69       123
         neu       0.52      0.64      0.57       183
         hap       0.84      0.68      0.75       232
         fru       0.61      0.63      0.62       184
         ang       0.67      0.64      0.65        88

    accuracy                           0.66       810
   macro avg       0.67      0.65      0.66       810
weighted avg       0.67      0.66      0.66       810



# Bias Analysis

In [74]:
# You are NOT allowed to use off-the-shelf fairness packages like ai360

def stat_parity(preds, sens):
    '''
    :preds: numpy array of the model predictions. Consisting of 0s and 1s
    :sens: numpy array of the sensitive features. Consisting of 0s and 1s
    :return: the statistical parity. no need to take the absolute value
    '''
    
    
    df = pd.DataFrame({'pred':preds,'prot':sens})

    unpriv = len(df[(df['pred']==1)&(df['prot']==0)])/len(df[df['prot']==0])
    priv = len(df[(df['pred']==1)&(df['prot']==1)])/len(df[df['prot']==1])
    return priv - unpriv

    
    # TODO. 10pts


def eq_oppo(preds, sens, labels):
    tpr_1,tpr_2 = 0,0
    '''
    :preds: numpy array of the model predictions. Consisting of 0s and 1s
    :sens: numpy array of the sensitive features. Consisting of 0s and 1s
    :labels: numpy array of the ground truth labels of the outcome. Consisting of 0s and 1s
    :return: the statistical parity. no need to take the absolute value
    '''
    df = pd.DataFrame({'pred':preds,'prot':sens,'labels':labels})
    try:
        a = len(df[(df['pred']==1) & (df['labels']==1) & (df['prot']==1)]) / len(df[(df['labels']==1) & (df['prot']==1)])
    except:
        a = 0
    try:
        b = len(df[(df['pred']==1) & (df['labels']==1) & (df['prot']==0)]) / len(df[(df['labels']==1) & (df['prot']==0)])
    except:
        b=0    
    
    return a-b

In [84]:
dc = {'F':0,'M':1}
sens = df_test['Gender'].apply(lambda a: dc[a])
labels = df_test['label']
# preds = preds
bias_df = pd.DataFrame({'gender':sens,'label':labels,'pred':preds})

In [90]:
bias_df['gender'].value_counts()

0    425
1    385
Name: gender, dtype: int64

In [92]:
for b_l in bias_df['label'].unique():
    print(b_l)
    break

0


# Emotion Wise Classification Report

In [119]:
temp_df = bias_df[bias_df['gender']==0]

print("FEMALE")
print(classification_report(temp_df['label'], temp_df['pred'], target_names=emo_ls))

FEMALE
              precision    recall  f1-score   support

         sad       0.67      0.65      0.66        69
         neu       0.46      0.59      0.51        92
         hap       0.85      0.65      0.74       125
         fru       0.61      0.65      0.63       100
         ang       0.64      0.64      0.64        39

    accuracy                           0.64       425
   macro avg       0.65      0.64      0.64       425
weighted avg       0.66      0.64      0.64       425



In [120]:
temp_df = bias_df[bias_df['gender']==1]

print("MALE")

print(classification_report(temp_df['label'], temp_df['pred'], target_names=emo_ls))

MALE
              precision    recall  f1-score   support

         sad       0.71      0.74      0.73        54
         neu       0.58      0.69      0.63        91
         hap       0.84      0.71      0.77       107
         fru       0.61      0.61      0.61        84
         ang       0.69      0.63      0.66        49

    accuracy                           0.68       385
   macro avg       0.68      0.68      0.68       385
weighted avg       0.69      0.68      0.68       385



# Statistical Parity & Equal Opportunity

In [118]:
bias_df

for emo in range(0,5):
    print(emo_ls[emo],end= "\t")
    sens,labels,preds = bias_df['gender'],bias_df['label'].apply(lambda a:0 if a==emo else 1),bias_df['pred'].apply(lambda a:0 if a==emo else 1)

    print(eq_oppo(preds, sens, labels), stat_parity(preds, sens))
    print()


sad	0.01345938422892834 0.01219251336898386

neu	0.03572960715817863 -0.005469824293353742

hap	-0.007290167865707464 -0.012834224598930466

fru	0.01651929465882951 0.03122994652406419

ang	-0.00539723661485314 -0.02511841100076395



In [123]:
[int(i==j) for i,j in zip(bias_df['label'],bias_df['pred'])]

[0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,


In [102]:
bias_df['label'].apply(lambda a:0 if a==0 else 1).value_counts()

1    687
0    123
Name: label, dtype: int64

In [104]:
bias_df['label'].value_counts()

2    232
3    184
1    183
0    123
4     88
Name: label, dtype: int64

In [ ]:
sens

In [81]:
preds

[2,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 4,
 4,
 4,
 0,
 1,
 1,
 3,
 1,
 3,
 3,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 3,
 2,
 0,
 1,
 3,
 0,
 2,
 1,
 2,
 3,
 3,
 3,
 3,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 3,
 3,
 1,
 1,
 2,
 1,
 4,
 3,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 3,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 3,
 4,
 4,
 4,
 3,
 1,
 0,
 0,
 3,
 3,
 4,
 3,
 1,
 3,
 3,
 3,
 1,
 4,
 3,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 3,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 1,
 1,
 3,
 1,
 3,
 2,
 3,
 4,
 1,
 0,
 3,
 4,
 1,
 1,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 4,
 4,
 4,
 3,
 0,
 4,
 1,
 3,
 3,
 4,
 0,
 0,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 0,
 0,
 3,
 3,


In [73]:
df_train['label']

0       0
1       0
2       0
3       0
4       0
       ..
7375    3
7376    3
7377    3
7378    3
7379    4
Name: label, Length: 5758, dtype: int64

In [71]:
preds

[2,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 4,
 4,
 4,
 0,
 1,
 1,
 3,
 1,
 3,
 3,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 3,
 2,
 0,
 1,
 3,
 0,
 2,
 1,
 2,
 3,
 3,
 3,
 3,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 3,
 3,
 1,
 1,
 2,
 1,
 4,
 3,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 3,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 3,
 4,
 4,
 4,
 3,
 1,
 0,
 0,
 3,
 3,
 4,
 3,
 1,
 3,
 3,
 3,
 1,
 4,
 3,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 3,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 1,
 1,
 3,
 1,
 3,
 2,
 3,
 4,
 1,
 0,
 3,
 4,
 1,
 1,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 4,
 4,
 4,
 3,
 0,
 4,
 1,
 3,
 3,
 4,
 0,
 0,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 0,
 0,
 3,
 3,


# Saving Embeddings

In [144]:
import torch
from tqdm.auto import tqdm

In [274]:

def get_sentence_embedding(sent):
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")

    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

    with torch.no_grad():
         output = model_embed(**encoded)


    hidden_states = output['hidden_states']
#     print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
    layer_i = 0

#     print ("Number of batches:", len(hidden_states[layer_i]))
    batch_i = 0

#     print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
    token_i = 0

#     print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

    token_embeddings = torch.stack(hidden_states, dim=0)
    
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)

#     print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

    # `hidden_states` has shape [13 x 1 x 22 x 768]

    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = hidden_states[-2][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)

#     print ("Our final sentence embedding vector of shape:", sentence_embedding.size())
    return sentence_embedding


# token_embeddings.size()

In [278]:
model_embed = AutoModelForSequenceClassification.from_pretrained(
    './test-trainer2/checkpoint-2000',
    output_hidden_states = True
)
embed_dict = pickle.load(open('../data/dump_embeddings.pkl','rb'))

In [286]:
embed_dict['text'] = list(df_train['text'])+list(df_test['text'])+list(df_valid['text'])

In [289]:
model_embed.eval()
embeds =[]
for sent in tqdm(embed_dict['text']):
    embeds.append(get_sentence_embedding(sent))

  0%|          | 0/7380 [00:00<?, ?it/s]

In [290]:
embed_dict['embeddings'] = embeds

In [291]:
with open('../data/dump_embeddings.pkl','wb+') as f:
    pickle.dump(embed_dict,f)

In [297]:
sum(embeds[0])

tensor(22.6907)

In [311]:
df = pd.DataFrame(embed_dict)

In [317]:
embed_dict['embeddings'][0]

tensor([-2.5028e-01, -2.7225e-02,  1.7844e-01,  5.5549e-01,  1.1923e+00,
         9.2259e-01, -1.0498e+00,  2.0631e-01,  1.0984e+00,  1.7576e-01,
        -1.4359e-01, -1.0987e+00, -2.7227e-01, -1.0615e+00,  1.0146e+00,
         1.0194e-01, -1.3428e+00, -3.2196e-01,  1.3269e-02,  9.1509e-01,
        -1.3659e+00, -3.3974e-01, -2.5787e-01, -4.4873e-01,  2.6469e-01,
         1.1638e+00, -6.3356e-01, -2.4429e-01,  1.6176e+00,  2.3976e-02,
        -3.9200e-01,  6.2035e-01, -1.7817e-01, -7.0268e-01,  6.7816e-01,
         2.1686e-01,  1.8320e-01, -1.6528e-01, -9.0290e-02, -2.3172e-01,
         1.2763e-01, -8.9797e-01, -2.6279e-01, -1.3188e+00, -2.4802e-02,
        -3.5177e-01,  8.8172e-01,  2.9082e-01,  1.9420e-01,  8.6537e-01,
         1.8325e-01,  1.6114e-01,  8.1260e-01, -1.7920e-01,  2.7502e-01,
         5.8187e-01, -5.7841e-01,  5.0530e-01, -7.5623e-01, -5.7425e-02,
         2.1691e-01,  4.3689e-01, -1.3669e-01,  1.9610e-01,  3.0762e-01,
        -1.4486e-01,  1.1787e+00, -9.3914e-01, -5.3

In [310]:
!ls -ltrh ../data/dump_embeddings.pkl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
-rw-rw-r--+ 1 rehanahm div21 28M Apr 10 04:02 ../data/dump_embeddings.pkl


tensor([ 7.1753e-01,  7.1740e-01,  1.9311e-01,  1.4703e-01,  4.0916e-01,
         1.1628e+00,  6.6178e-01,  2.3801e-02, -9.9653e-02, -6.0544e-02,
        -2.2448e-01,  1.7581e-01,  6.5957e-01, -4.0459e-01, -9.6910e-02,
        -6.9616e-01,  4.5281e-01, -8.6157e-01, -1.3240e-02,  2.0969e+00,
         3.1397e-02, -1.3169e+00, -5.7897e-01, -4.5992e-01, -1.5242e+00,
         1.4780e-01,  1.7647e-01,  3.9375e-02,  1.5078e-01,  1.7738e+00,
         1.5572e-01, -7.6408e-01,  2.8126e-01,  2.4109e-01, -3.0096e-01,
         2.0284e-01, -3.2424e-01,  7.9904e-01,  1.1046e+00, -8.2546e-01,
         5.4125e-01,  6.3639e-01,  2.7381e-01,  8.4687e-01,  5.0385e-01,
        -8.5032e-01,  6.1866e-02,  1.1222e+00,  1.5168e-01, -7.1361e-01,
        -2.1125e-01,  7.3605e-01,  4.0123e-01, -4.7860e-01,  1.9189e+00,
        -7.0153e-01, -3.0373e-01, -3.9389e-01, -5.4133e-01,  7.1282e-01,
        -3.8879e-01,  1.1574e+00,  1.1314e+00, -1.6852e-02,  6.9457e-01,
         4.7828e-01, -2.4999e-01, -1.7124e+00,  2.8

In [ ]:
for i in 

In [273]:
get_sentence_embedding(df.iloc[0]['text'])

Shape is: 13 x 768
Our final sentence embedding vector of shape: torch.Size([768])


tensor([-2.5028e-01, -2.7225e-02,  1.7844e-01,  5.5549e-01,  1.1923e+00,
         9.2259e-01, -1.0498e+00,  2.0631e-01,  1.0984e+00,  1.7576e-01,
        -1.4359e-01, -1.0987e+00, -2.7227e-01, -1.0615e+00,  1.0146e+00,
         1.0194e-01, -1.3428e+00, -3.2196e-01,  1.3269e-02,  9.1509e-01,
        -1.3659e+00, -3.3974e-01, -2.5787e-01, -4.4873e-01,  2.6469e-01,
         1.1638e+00, -6.3356e-01, -2.4429e-01,  1.6176e+00,  2.3976e-02,
        -3.9200e-01,  6.2035e-01, -1.7817e-01, -7.0268e-01,  6.7816e-01,
         2.1686e-01,  1.8320e-01, -1.6528e-01, -9.0290e-02, -2.3172e-01,
         1.2763e-01, -8.9797e-01, -2.6279e-01, -1.3188e+00, -2.4802e-02,
        -3.5177e-01,  8.8172e-01,  2.9082e-01,  1.9420e-01,  8.6537e-01,
         1.8325e-01,  1.6114e-01,  8.1260e-01, -1.7920e-01,  2.7502e-01,
         5.8187e-01, -5.7841e-01,  5.0530e-01, -7.5623e-01, -5.7425e-02,
         2.1691e-01,  4.3689e-01, -1.3669e-01,  1.9610e-01,  3.0762e-01,
        -1.4486e-01,  1.1787e+00, -9.3914e-01, -5.3

In [ ]:
model_embed.eval()

In [256]:
sent = df_train.iloc[10]['text']

encoded = tokenizer.encode_plus(sent, return_tensors="pt")

token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

with torch.no_grad():
     output = model_embed(**encoded)


hidden_states = output['hidden_states']

In [257]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 31
Number of hidden units: 768


In [258]:
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 31, 768])

In [259]:
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 31, 768])

In [260]:
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([31, 13, 768])

In [261]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Shape is: 31 x 768


In [263]:
sentence_embedding

tensor([ 7.9241e-01,  7.9641e-01, -2.0244e-01, -6.0815e-02,  9.8484e-02,
         1.0083e+00,  1.7977e-01,  2.9316e-01, -2.3318e-01, -1.9373e-02,
        -5.4172e-01,  1.1561e-01,  1.0236e+00, -4.7100e-01,  7.5856e-02,
        -1.0722e+00,  4.7360e-01, -7.9421e-01,  6.4980e-01,  1.7483e+00,
        -2.8437e-01, -1.2566e+00, -2.4930e-01, -2.5476e-01, -1.4871e+00,
        -1.7104e-01, -2.3814e-01,  4.2275e-01,  4.2473e-01,  1.4769e+00,
         3.7531e-01, -7.5099e-01,  5.7870e-01, -4.7627e-01, -2.6753e-01,
         6.5057e-01,  1.9836e-02,  1.4142e-01,  2.6741e-01, -9.7723e-01,
         1.0625e-01,  9.1154e-02,  1.0801e+00,  8.9258e-01,  8.3028e-01,
        -5.8031e-01, -1.9542e-01,  1.3636e+00, -5.3820e-02, -7.7757e-01,
        -4.9669e-02,  5.9412e-01,  4.6147e-01, -5.3151e-01,  1.4766e+00,
        -3.6486e-01,  4.1003e-02, -3.5144e-01, -4.9082e-01,  5.1891e-01,
        -1.4226e-01,  6.0131e-01,  1.0701e+00,  1.4234e-01,  6.9941e-01,
         4.8953e-01, -7.3775e-01, -1.4916e+00,  4.0

In [255]:
sentence_embedding

tensor([ 8.4470e-01,  7.1697e-01,  2.5688e-02, -1.6738e-01,  2.3583e-01,
         6.6787e-01, -1.3944e-01,  1.3767e-01, -1.3260e-01, -1.4556e-01,
        -3.5052e-01,  4.7551e-01,  7.1150e-01, -1.2456e-01,  2.3886e-01,
        -1.2112e+00,  2.1870e-01, -6.2340e-01,  7.4428e-01,  1.7221e+00,
        -3.4774e-01, -9.0149e-01, -3.3343e-01, -4.8426e-01, -1.1375e+00,
        -1.6021e-01, -4.5457e-01,  6.6641e-01,  3.0233e-01,  1.5861e+00,
         5.3190e-01, -3.3844e-01,  3.1331e-01, -6.6308e-01, -2.0351e-01,
         8.9602e-01, -1.7156e-01,  1.5755e-01,  6.3668e-01, -8.1655e-01,
         5.4786e-01,  2.0781e-02,  1.3678e+00,  7.6663e-01,  8.0346e-01,
        -1.3833e-01, -3.5393e-02,  1.5194e+00, -1.7736e-01, -5.7622e-01,
        -1.9066e-01,  5.5848e-01,  2.2112e-01, -9.2591e-01,  1.0626e+00,
        -5.1608e-01,  8.2437e-02, -4.1658e-01, -3.2709e-01,  5.6486e-01,
        -1.0597e-01,  5.9780e-01,  6.6816e-01, -1.4532e-01,  6.0984e-01,
         6.2301e-01, -9.8247e-01, -1.9989e+00,  2.6

In [227]:
output[0]

tensor([[-0.6246,  1.2481,  5.7773, -1.0492, -1.8568, -2.8500, -2.5932]])

In [200]:
len(output['hidden_states'])

13

In [206]:
output['hidden_states']

(tensor([[[ 1.6518e-01, -4.8222e-02,  1.4864e-04,  ..., -7.6252e-02,
            8.4043e-02,  1.3102e-02],
          [-6.0285e-02, -3.1149e-01,  1.5629e-01,  ...,  1.0649e-02,
            1.2823e-01,  2.6586e-01],
          [ 1.5781e-01, -4.8849e-03, -3.7191e-02,  ...,  2.6678e-01,
           -3.5979e-01,  4.6399e-02],
          ...,
          [ 9.7358e-01, -6.6068e-01, -3.9619e-01,  ...,  2.9260e-01,
            1.6849e-01,  3.7391e-01],
          [ 5.4959e-01, -2.7490e-01,  1.7637e-01,  ..., -1.1539e-01,
            2.9791e-02,  2.7837e-01],
          [ 1.6472e-01, -1.5221e-01,  5.0608e-02,  ...,  3.8052e-01,
            1.5168e-01,  1.2706e-01]]]),
 tensor([[[ 1.3622e-02,  6.2574e-02, -9.9679e-03,  ..., -8.2492e-03,
            8.5152e-04, -8.5083e-02],
          [-2.2514e-01, -7.8329e-01,  2.2409e-01,  ..., -6.1050e-01,
            9.5662e-02,  7.0830e-01],
          [-7.3169e-02, -4.8602e-02, -1.2962e-02,  ...,  1.3357e-01,
           -5.8045e-01,  5.1867e-01],
          ...,
    

In [215]:
output['hidden_states'][0][0][0].shape

torch.Size([768])

In [216]:
output['hidden_states'][0][0][1].shape

torch.Size([768])

In [220]:
len(output['hidden_states'])

13

In [221]:
8*13

104

In [207]:
len(output['hidden_states'][0])

1

torch.Size([1, 7])

In [162]:
states

In [135]:


# def get_word_vector(sent, idx, tokenizer, model, layers):
encoded = tokenizer.encode_plus(sent, return_tensors="pt")
 # get all token idxs that belong to the word of interest
token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
 
with torch.no_grad():
     output = model(**encoded)

# Get all hidden states
states = output.hidden_states
# Stack and sum all requested layers
output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
# Only select the tokens that constitute the requested word
word_tokens_output = output[token_ids_word]

word_tokens_output.mean(dim=0)
#     return get_hidden_states(encoded, token_ids_word, model, layers)

[0,
 33082,
 4,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [131]:
embed_dict['input_ids'][1]

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]